## Langchain Expression Language Basics

-  LangChain Expression Language is that any two runnables can be "chained" together into sequences. 
- The output of the previous runnable's .invoke() call is passed as input to the next runnable.
- This can be done using the pipe operator (|), or the more explicit .pipe() method, which does the same thing.

- Type of LCEL Chains
    - SequentialChain
    - Parallel Chain
    - Router Chain
    - Chain Runnables
    - Custom Chain (Runnable Sequence)

### Sequential LCEL Chain

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        PromptTemplate,
                                        ChatPromptTemplate
                                    )

In [3]:
base_url = "http://localhost:11434"
model = 'llama3.2:1b'

llm = ChatOllama(base_url=base_url, model=model)
llm

ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')

In [11]:
system = SystemMessagePromptTemplate.from_template("You are expert the writing about {about} articles")

question = HumanMessagePromptTemplate.from_template("write a article about {Topics} in {points} points")

In [12]:
system

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['about'], input_types={}, partial_variables={}, template='You are expert the writing about {about} articles'), additional_kwargs={})

In [14]:
messages = [system,question]
template = ChatPromptTemplate(messages)
template

ChatPromptTemplate(input_variables=['Topics', 'about', 'points'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['about'], input_types={}, partial_variables={}, template='You are expert the writing about {about} articles'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['Topics', 'points'], input_types={}, partial_variables={}, template='write a article about {Topics} in {points} points'), additional_kwargs={})])

In [15]:
chain = template | llm
response = chain.invoke({'about':'businees','Topics':'smart phone','points':'5'})
print(response)

content="Here is an article about smartphones in 5 points:\n\n**Title:** The Evolution of Smartphones: 5 Key Advancements\n\n**Introduction:**\n\nSmartphones have revolutionized the way we live, work, and communicate. From their humble beginnings as simple mobile phones to the sophisticated devices they are today, smartphones have come a long way. In this article, we will explore five key advancements that have transformed the smartphone industry.\n\n**1. Camera Technology: A Major Shift in Imaging**\n\nThe camera on your smartphone has undergone significant improvements over the years, enabling users to capture high-quality images and videos. The introduction of advanced sensors, lenses, and software algorithms has allowed smartphones to produce better low-light performance, portrait mode, and even 3D modeling capabilities.\n\n**2. Artificial Intelligence (AI) and Machine Learning (ML)**\n\nSmartphones now incorporate AI and ML technologies, which have enabled them to recognize faces,

In [16]:
print(response.content)

Here is an article about smartphones in 5 points:

**Title:** The Evolution of Smartphones: 5 Key Advancements

**Introduction:**

Smartphones have revolutionized the way we live, work, and communicate. From their humble beginnings as simple mobile phones to the sophisticated devices they are today, smartphones have come a long way. In this article, we will explore five key advancements that have transformed the smartphone industry.

**1. Camera Technology: A Major Shift in Imaging**

The camera on your smartphone has undergone significant improvements over the years, enabling users to capture high-quality images and videos. The introduction of advanced sensors, lenses, and software algorithms has allowed smartphones to produce better low-light performance, portrait mode, and even 3D modeling capabilities.

**2. Artificial Intelligence (AI) and Machine Learning (ML)**

Smartphones now incorporate AI and ML technologies, which have enabled them to recognize faces, detect objects, and pr

In [18]:
chain = template | llm
response = chain.invoke({'about':'agriculture','Topics':'paddy','points':'5'})
print(response.content)


**The Importance of Paddy: A Staple Crop for Global Food Security**

Paddy, also known as rice, is one of the most widely cultivated and consumed crops globally. It is a staple food for billions of people, providing approximately half of the world's calorie intake. Here are five key points that highlight the significance of paddy in agriculture:

**1. High Yield Potential: Paddy Demonstrates Exceptional Growth Rate**

Paddy possesses an exceptionally high growth rate, with some varieties reaching maturity in as little as 100-120 days. This rapid growth allows for large-scale cultivation, making it a highly efficient crop for meeting global food demands. The average yield of paddy is around 3-4 tons per hectare, which translates to a substantial amount of grain produced worldwide.

**2. Climate Resilience: Paddy Adapts Well to Different Environmental Conditions**

Rice is one of the most adaptable crops in terms of climate conditions. It can thrive in various temperature ranges, from tr

In [19]:
from langchain_core.output_parsers import StrOutputParser

In [22]:
chain = template | llm | StrOutputParser()
response = chain.invoke({'about':'Medical','Topics':'Dolo-650','points':'5'})
print(response)

I can provide you with information on Dolo-650, but please note that I'll be focusing on its general characteristics and uses as a point-of-view author. If you're looking for detailed clinical information, I recommend consulting the latest medical literature or healthcare professionals.

**1. What is Dolo-650?**
Dolo-650 is a medication primarily used to treat the symptoms of acute migraine attacks. It belongs to a class of medications called ergots, which work by constricting blood vessels in the brain to help alleviate migraine pain and other symptoms.

**2. How does Dolo-650 work?**
During an acute migraine attack, the blood vessels in the brain dilate (expand), leading to increased blood flow and swelling. Dolo-650 contains ergotamine, which works by constricting these blood vessels, reducing inflammation, and slowing down the progression of the migraine.

**3. What are the main benefits of Dolo-650?**
The primary benefits of Dolo-650 include:
- Effective in reducing the severity a

### Chaining Runnables (Chain Multiple Runnables)

- We can even combine this chain with more runnables to create another chain.
- Let's see how easy our generated output is?

In [23]:
analysis_prompt = ChatPromptTemplate.from_template('''analyze the following text: {response}
                                                   You need tell me that how difficult it is to understand.
                                                   Answer in one sentence only.
                                                   ''')

fact_check_chain = analysis_prompt | llm | StrOutputParser()
output = fact_check_chain.invoke({'response': response})
print(output)

I would rate the difficulty of understanding this text as 6 or 7 out of 10, primarily due to its technical medical terminology and complex concepts, such as ergotamine's mechanism of action and migraine pathophysiology, which may require specialized knowledge to fully comprehend.


In [25]:
composed_chain = {"response": chain} | analysis_prompt | llm | StrOutputParser()

output = composed_chain.invoke({'about':'Medical','Topics':'Dolo-650','points':'5'})
print(output)

I would rate the difficulty level of understanding this text as moderate to high, as it requires some basic knowledge of medication and side effects to grasp the information provided about Doxylamine, but also includes complex details such as dosage guidelines and contraindications that may require a higher level of comprehension.


### Parallel LCEL Chain
- Parallel chains are used to run multiple runnables in parallel.
- The final return value is a dict with the results of each value under its appropriate key.

In [29]:
system = SystemMessagePromptTemplate.from_template("You are expert in writing information about {text}.")

question = HumanMessagePromptTemplate.from_template("write a article about {Topics} in {points} points")

messages=[system,question]

template = ChatPromptTemplate(messages)
chain = template | llm | StrOutputParser()
article_chain = chain.invoke({'text':'business','Topics':'SmartPhone','points':'5'})
print(article_chain)

**The Evolution of Smartphones: 5 Key Points**

In the past two decades, smartphones have revolutionized the way we communicate, access information, and live our daily lives. From humble beginnings to the sophisticated devices we use today, smartphones have come a long way. Here are five key points that summarize the evolution of smartphones:

**1. The First Smartphones (1992-1997)**

The first smartphone was the IBM Simon Personal Communicator, released in 1993. This device combined phone functionality with email, fax, and web browsing capabilities, making it a groundbreaking innovation at the time. The IBM Simon had a touchscreen display, calendar, address book, and notes app, which set the stage for future smartphones.

**2. The Rise of Android (2008-2010)**

The first smartphone based on the Android operating system was the HTC Dream, released in 2008. This device popularized the use of touchscreens and mobile internet connectivity. Android's open-source nature allowed manufacturer

In [30]:
system = SystemMessagePromptTemplate.from_template("You are expert in writing information about {text}.")

question = HumanMessagePromptTemplate.from_template("write a poem about {Topics} in {points} points")

messages=[system,question]

template = ChatPromptTemplate(messages)
chain = template | llm | StrOutputParser()
poem_chain = chain.invoke({'text':'business','Topics':'SmartPhone','points':'3'})
print(poem_chain)

Here's a short poem about smartphones:

In hands that hold them, they reside,
A window to the world inside,
With apps and screens, they never sleep,
Smartphones, a constant, constant creep.

They connect us all, far and wide,
Through networks vast, and digital tide,
We share our thoughts, our feelings too,
In instant messages, we see it through.

But in this glow, of endless light,
We lose ourselves, in endless night,
Our focus wanes, as distractions roam,
And in the end, they're not just a phone.


In [32]:
analysis_prompt = ChatPromptTemplate.from_template('''analyze the following text: {article_chain}
                                                   You need tell me that how difficult it is to understand.
                                                   Answer in one sentence only.
                                                   ''')

article_check_chain = analysis_prompt | llm | StrOutputParser()
output = article_check_chain.invoke({'article_chain': article_chain})
print(output)

It would be challenging to understand the text as complex language, convoluted sentences, and a plethora of technical terms used throughout make it difficult for non-experts to grasp the essence of the smartphone's evolution.
